## STEP1 Variableクラスの実装

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

#### Test & memo  
 = no.array() 多次元配列も可能としている。  
 列数が次元数。行数がデータ数

In [2]:
import numpy as np

data = np.array([[1,2,3],[4,5,6,],])
x = Variable(data)
print(x.data)

[[1 2 3]
 [4 5 6]]


## STEP2 Functionクラスの実装
y = f(x) 　　x,yはVariableとする。  
- Functionクラスは基底クラスとし、すべての関数に共通する機能を実装する。
- 具体的な関数は、Functionクラスを継承したクラスで実装する。

In [3]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)    # 具体的な計算は、forwardメソッドでで行う。
        return Variable(y)
    
    def forward(self, x):
        raise NoImplementedError()

In [4]:
class Squre(Function):
    def forward(self, x):
        return x ** 2

#### Test & Memo  


In [5]:
x = Variable(np.array(10))
f = Squre()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## STEP3 関数の連結
### Exp関数の実装

In [6]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

#### Test & Memo
- 変数がすべて同一の型（Variableインスタンス）である点が重要
- 合成関数、計算グラフで表現することでバックプロパゲーション（誤差伝播法）のアルゴリズムを可能にする。

In [7]:
A = Squre()
B = Exp()
C = Squre()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## STEP4 数値微分
- 数値微分に代わるより効率的なアルゴリズムとしてのバックプロパゲーション
- 中心差分近似　$\frac{f(x+h)-f(x-h)}{2h}$の関数（メソッド）をnumerical_diff(f, x, eps=1e-4)と定義

In [8]:
def numerical_deff(f, x, eps=1e-4):
    x0 = Variable(x.data -eps)
    x1 = Variable(x.data +eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

#### Test & Memo
誤差を含まない正確な微分値が4.0なのに対し、下の近似は十分に良い値と言える。

In [9]:
f = Squre()
x = Variable(np.array(2.0))
dy = numerical_deff(f, x)
print(dy)

4.000000000004


### 合成関数の微分
合成関数　$ y=(e^{x^2})^2 $　の微分 $ \frac{dy}{dx} $を求める。

In [10]:
def f(x):
    A = Squre()
    B = Exp()
    C = Squre()
    return C(B(A(x)))

#### Test & Memo
- どんなに複雑な組み合わせの関数であっても、微分可能な関数であるなら自動微分できる。

In [11]:
x = Variable(np.array(0.5))
dy = numerical_deff(f, x)
print(dy)

3.2974426293330694


## STEP5 バックプロパゲーションの理論
- チェインルール（連鎖律）：合成関数の微分は、それを構成する各関数の部分の積に分解できること。
- ここで、積の順番は入れ替え可能。つまり逆向き（yの〇に関する微分）に表現できる。（逆伝播）
- バックプロパゲーションの実装では、はじめに順伝播を行い、各変数の値を記憶しておく。それにより、各関数の逆伝播の計算が可能になる。


## STEP6 手作業によるバックプロパゲーション
##### Variableクラスの改造
    - 通常の値(data)と微分した値(grad)を保持（ともに多次元配列 ndarray）
    - gradは、Noneで初期化し、逆伝播で微分が計算されたときに設定する。
##### Functionの改造
    - 逆伝播の機能（backwardメソッド）の実装
    - forward呼び出し時に、入力値を保持する機能

In [12]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

In [13]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input    # 入力値を保持する。backwardメソッド呼び出し時に使用する。
        return output
    
    def forward(self, x):
        raise NotImplementError()
        
    def backward(self, gy):
        raise NotImplementedError()    

##### SqureクラスとExpクラスの改造
- $ y=x^2 $の微分は、$ \frac{dy}{dx}=2x $であること。
- $ y=e^x $の微分は、$ \frac{dy}{dx}=e^x $であること。

In [14]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [15]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

##### 逆伝播をテストする
まずは、順伝播を記述し、テストする。

In [16]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [17]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## STEP7 バックプロパゲーションの自動化　Define-by-Run

In [18]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func